In [22]:
!pip install matplotlib
!pip install pymysql sqlalchemy
!pip install mysql-connector-python python-dotenv


In [23]:
import os

from dotenv import load_dotenv

class Config:
    def __init__(self):
        load_dotenv()
        self.tidb_host = os.getenv("TIDB_HOST", "gateway01.us-east-1.prod.aws.tidbcloud.com")
        self.tidb_port = int(os.getenv("TIDB_PORT", "4000"))
        self.tidb_user = os.getenv("TIDB_USER", "EcFsmzHzn16sz32.root")
        self.tidb_password = os.getenv("TIDB_PASSWORD", "4UTXzVBKxU10w2Z1")
       # self.tidb_db_name = os.getenv("TIDB_DB_NAME", "embracepath")
        self.tidb_db_name = "embracepath"
        self.ca_path = os.getenv("CA_PATH", "")

In [35]:
# db_service.py
import mysql.connector
from mysql.connector import MySQLConnection
import logging
import pandas as pd

class DatabaseService:
    def __init__(self):
        self.config = Config()
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    def get_connection(self, autocommit: bool = True) -> MySQLConnection:
        db_conf = {
            "host": self.config.tidb_host,
            "port": self.config.tidb_port,
            "user": self.config.tidb_user,
            "password": self.config.tidb_password,
            "database": self.config.tidb_db_name,
            "autocommit": autocommit,
            "use_pure": True,
        }

        if self.config.ca_path:
            db_conf["ssl_verify_cert"] = True
            db_conf["ssl_verify_identity"] = True
            db_conf["ssl_ca"] = self.config.ca_path

        try:
            connection = mysql.connector.connect(**db_conf)
            return connection
        except mysql.connector.Error as e:
            logging.error("Error connecting to MySQL Platform: %s", e)
            return None
    
    def fetch_data(self, query: str, params: tuple = None) -> pd.DataFrame:
        try:
            with self.get_connection() as conn:
                if params is not None:
                    return pd.read_sql(query, conn, params=params)
                else:
                    return pd.read_sql(query, conn)
        except Exception as e:
            logging.error("Failed to fetch data: %s", e)
            return pd.DataFrame()  # Return an empty DataFrame on failure

        
        
    def get_reward(self, state, exercise_id: int, emotion_id: int) -> float:
        query = """
        SELECT feedback FROM user_exercise_feedback 
        WHERE user_id = %s AND exercise_id = %s AND emotion_id = %s
        """
        feedback_df = self.fetch_data(query, (state.user_id, exercise_id, emotion_id))
        if not feedback_df.empty:
            return feedback_df['feedback'].values[0] / 5
        else:
            return 0.0  # or some default value

        
        
            
    def fetch_num_exercises(self):
        query = "SELECT COUNT(*) FROM recommended_exercise WHERE active_flg = TRUE"
        df = self.fetch_data(query)
        if not df.empty:
            return df.iloc[0, 0]
        return 0
            
    def fetch_recommended_exercises(self):
        query = """
        SELECT recommended_exercise_id, exercise_id, initial_q_value
        FROM recommended_exercise
        WHERE active_flg = TRUE
        """
        return self.fetch_data(query)
    
    def get_emotion_vector_by_typename(self, typeName: str):
        query = """
        USE embracepath;
        SELECT 
        TypeID,
        TypeName,
        TypeNameVector,
        Description,
        DescriptionVector,
        create_by,
        create_dt,
        modified_by,
        modified_dt,
        active_flg
        FROM embracepath.lu_emotion_type WHERE TypeName = %s
        """
        return self.fetch_data(query, params=(typeName,))


In [25]:
class EpsilonGreedyAgent:
    def __init__(self, epsilon, environment):
        self.epsilon = epsilon
        self.environment = environment
        self.exercises_df = environment._fetch_recommended_exercises()
        self.Q_values = np.zeros(len(self.exercises_df))  # Adjust size based on fetched data

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            # Explore: choose a random action from the available exercises
            action_idx = np.random.randint(len(self.exercises_df))
        else:
            # Exploit: choose the best known action
            action_idx = np.argmax(self.Q_values)
        action = self.exercises_df.iloc[action_idx]
        return (action['recommended_exercise_id'], action['exercise_id'], action['initial_q_value'])

    def update(self, action_idx, reward):
        # Incremental update to the Q-value for the chosen action
        self.Q_values[action_idx] += (reward - self.Q_values[action_idx]) / (self.environment.action_counts[action_idx] + 1)
        self.environment.action_counts[action_idx] += 1


In [26]:
class State:
    def __init__(self, core_emotion, user_id, emotion_vector):
        self.core_emotion = core_emotion
        self.user_id = user_id
        self.emotion_vector = emotion_vector

In [27]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box
import numpy as np

class MindfulnessBanditEnv(gym.Env):
    def __init__(self, num_features:int, db_service:DatabaseService, num_actions:int):
        self.num_features = num_features
        self.db_service = db_service
        self.action_space = Discrete(num_actions)  # 10 possible actions (mindfulness exercises)
        self.observation_space = Box(low=0, high=1, shape=(num_features,), dtype=np.float32)
        self.state = None
        self.done = False

    def reset(self):
        core_emotion = 1 # Happiness
        user_id = 1
        emotion_vector = self.db_service.get_emotion_vector_by_typename("Happiness")

        emotion_vector = self.db_service.get_emotion_vector_by_typename("Happiness")
        self.state = State(core_emotion, user_id, emotion_vector)
        observation = np.array([self.state.core_emotion, self.state.user_id, *self.state.emotion_vector]) # * is times the number of dimensions in the shape. in this it is 3
        return observation

    def step(self, action):
        # Simulate the delay between the selected mindfulness exercise and the user's rating
        rating = self.get_user_rating(action)
        reward = self.db_service.get_reward(self.state, action, rating)
        self.done = True
        return self.state, reward, self.done, {}

    def get_user_rating(self, action):
        # Simulate the user's rating (1-5)
        rating = np.random.randint(1, 6)
        return rating

    def close(self):
        pass

In [36]:
# Create environment and agent
# Function to plot the learning history
def plot_learning_history(history):
    fig = plt.figure(1, figsize=(14, 10))
    ax = fig.add_subplot(2, 1, 1)
    episodes = np.arange(len(history))
    moves = np.array([h[0] for h in history])
    plt.plot(episodes, moves, lw=4, marker='o', markersize=10)
    ax.tick_params(axis='both', which='major', labelsize=15)
    plt.xlabel('Episodes', size=20)
    plt.ylabel('# moves', size=20)
    ax = fig.add_subplot(2, 1, 2)
    rewards = np.array([h[1] for h in history])
    plt.step(episodes, rewards, lw=4)
    ax.tick_params(axis='both', which='major', labelsize=15)
    plt.xlabel('Episodes', size=20)
    plt.ylabel('Final rewards', size=20)
    plt.savefig('q-learning-history.png', dpi=300)
    plt.show()
    
def main():
    # https://www.meta.ai/c/a04b014a-ece0-4c51-9aa0-c51efb911727
    # for second set of eyes coding https://chatgpt.com/c/5264785b-a953-4241-b977-7a96b811d4bd
    db_service = DatabaseService()
    # The features are angry, sad, happy
    env = MindfulnessBanditEnv(num_features=3, db_service=db_service, num_actions=10)
    # so i need to create a custom state #     user_info = {
#         'thought_pattern': 0,  # "I'm not good enough"
#         'location': 0,  # "at home"
#         'core_emotion': 0  # "sadness"
#           'user_id': 123 $# user id
#     } this is what i will be passing in. 
    
    history = []
    for episode in range(5):  # Simulate 5 episodes
        state, _ = env.reset()
        action = np.random.randint(0, 10)  # Randomly choose an action
        next_state, reward, done, _ = env.step(action)
        history.append((episode, reward))
        print(f"Episode {episode}: Action {action}, Reward {reward}")
    env.close()
    plot_learning_history(history)    
    
main()    



# def main(): 
    
#     # Define the user's thought type - this will be user current state
#     user_info = {
#         'thought_pattern': 0,  # "I'm not good enough"
#         'location': 0,  # "at home"
#         'core_emotion': 0  # "sadness"
#           'user_id': 123 $# user id
#     }
#     num_features = 3 # number of input categories
#     num_actions = 100
#     env = MindfulnessBanditEnv(num_features, num_actions)
#     q_values = np.zeros(num_actions)
#     agent = EpsilonGreedyAgent(epsilon=0.1, environment=env)

#     history = []
#     alpha = 0.1
#     gamma = 0.9

#     for episode in range(1000):
#         state = env.reset()
#         action = agent.select_action(state)
#         print("action is", action)
#         next_state, reward, done, _ = env.step(action)
#         agent.update(action, reward, alpha, gamma)
#         history.append((action, reward))
#         if episode % 100 == 0:
#             print(f"Episode {episode}: Action {action}, Reward {reward}")

#     env.close()
#     plot_learning_history(history)

# main()    


/var/tmp/ipykernel_2464/102851041.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn, params=params)


ValueError: too many values to unpack (expected 2)